In [61]:
import pandas as pd

In [62]:
# 1. Load the data
df = pd.read_csv('../data/mindx test dataset.csv')

In [63]:
# 2. Define "Efficiency" (Fuel burned per Nautical Mile)
# Formula: Efficiency = Fuel / Distance
df['fuel_per_mile'] = df['fuel_consumption'] / df['distance']

In [64]:
# BASELINE 1: GLOBAL VOYAGE AVERAGES (For Analysis 1)
# Ue the mean of ALL rows to benchmark a single trip
global_avg_eff = df['fuel_per_mile'].mean()
global_avg_fuel = df['fuel_consumption'].mean()
global_avg_co2 = df['CO2_emissions'].mean()

print(f"--- GLOBAL FLEET BENCHMARKS (Per Voyage) ---")
print(f"Avg Efficiency: {global_avg_eff:.2f} kg/nm")
print(f"Avg Fuel/Voyage: {global_avg_fuel:.0f} kg")
print(f"Avg CO2/Voyage: {global_avg_co2:.0f} kg")
print("="*60 + "\n")

--- GLOBAL FLEET BENCHMARKS (Per Voyage) ---
Avg Efficiency: 28.64 kg/nm
Avg Fuel/Voyage: 4844 kg
Avg CO2/Voyage: 13365 kg



In [65]:
# Analysis 1: The "Acute" Anomaly (Bad Voyage)
# Filter for Calm weather to isolate mechanical/physical issues
calm_df = df[df['weather_conditions'] == 'Calm']
worst_voyage = calm_df.sort_values(by='fuel_per_mile', ascending=False).iloc[0]

In [66]:
# Calculate Deviations from Global Average
v_fuel_diff = ((worst_voyage['fuel_consumption'] - global_avg_fuel) / global_avg_fuel) * 100
v_co2_diff = ((worst_voyage['CO2_emissions'] - global_avg_co2) / global_avg_co2) * 100
v_eff_diff = ((worst_voyage['fuel_per_mile'] - global_avg_eff) / global_avg_eff) * 100

In [67]:
print(f"REPORT 1: ACUTE ANOMALY (Single Voyage Analysis)")
print(f"Vessel ID: {worst_voyage['ship_id']}")
print(f"Route: {worst_voyage['route_id']}")
print(f"Weather: {worst_voyage['weather_conditions']}")
print("-" * 60)
print(f"{'METRIC':<20} | {'VOYAGE VALUE':<12} | {'FLEET AVG':<12} | {'VARIANCE':<10}")
print("-" * 60)
print(f"{'Fuel Consumed':<20} | {worst_voyage['fuel_consumption']:<12.0f} | {global_avg_fuel:<12.0f} | {v_fuel_diff:+.1f}%")
print(f"{'CO2 Emissions':<20} | {worst_voyage['CO2_emissions']:<12.0f} | {global_avg_co2:<12.0f} | {v_co2_diff:+.1f}%")
print(f"{'Engine Eff (kg/nm)':<20} | {worst_voyage['fuel_per_mile']:<12.2f} | {global_avg_eff:<12.2f} | {v_eff_diff:+.1f}%")
print("-" * 60)

REPORT 1: ACUTE ANOMALY (Single Voyage Analysis)
Vessel ID: NG117
Route: Lagos-Apapa
Weather: Calm
------------------------------------------------------------
METRIC               | VOYAGE VALUE | FLEET AVG    | VARIANCE  
------------------------------------------------------------
Fuel Consumed        | 16008        | 4844         | +230.5%
CO2 Emissions        | 40638        | 13365        | +204.1%
Engine Eff (kg/nm)   | 49.91        | 28.64        | +74.2%
------------------------------------------------------------


In [68]:
# BASELINE 2: SHIP-LEVEL AVERAGES (For Analysis 2)
# We aggregate by ship FIRST, then take the average.
# Compare "Ship vs. Ship" not "Ship vs. Fleet".
ship_stats = df.groupby('ship_id').agg({
    'fuel_per_mile': 'mean',
    'fuel_consumption': 'mean',
    'CO2_emissions': 'mean'
}).reset_index()

In [69]:
# The average "Ship" performance
avg_ship_eff = ship_stats['fuel_per_mile'].mean()
avg_ship_fuel = ship_stats['fuel_consumption'].mean()
avg_ship_co2 = ship_stats['CO2_emissions'].mean()

In [70]:
# ANALYSIS 2: The "Aggregate" Anomaly (Bad Ship)
# Group by Ship ID and calculate average efficiency
worst_ship = ship_stats.sort_values(by='fuel_per_mile', ascending=False).iloc[0]

In [71]:
# Calculate Deviations from Ship Average
s_fuel_diff = ((worst_ship['fuel_consumption'] - avg_ship_fuel) / avg_ship_fuel) * 100
s_co2_diff = ((worst_ship['CO2_emissions'] - avg_ship_co2) / avg_ship_co2) * 100
s_eff_diff = ((worst_ship['fuel_per_mile'] - avg_ship_eff) / avg_ship_eff) * 100

In [72]:
print(f"REPORT 2: CHRONIC ANOMALY (Ship Aggregation Analysis)")
print(f"Vessel ID: {worst_ship['ship_id']}")
print("-" * 60)
print(f"{'METRIC':<20} | {'SHIP AVG':<12} | {'ALL SHIPS AVG':<12} | {'VARIANCE':<10}")
print("-" * 60)
print(f"{'Avg Fuel/Voyage':<20} | {worst_ship['fuel_consumption']:<12.0f} | {avg_ship_fuel:<12.0f} | {s_fuel_diff:+.1f}%")
print(f"{'Avg CO2/Voyage':<20} | {worst_ship['CO2_emissions']:<12.0f} | {avg_ship_co2:<12.0f} | {s_co2_diff:+.1f}%")
print(f"{'Avg Efficiency':<20} | {worst_ship['fuel_per_mile']:<12.2f} | {avg_ship_eff:<12.2f} | {s_eff_diff:+.1f}%")
print("-" * 60)

REPORT 2: CHRONIC ANOMALY (Ship Aggregation Analysis)
Vessel ID: NG108
------------------------------------------------------------
METRIC               | SHIP AVG     | ALL SHIPS AVG | VARIANCE  
------------------------------------------------------------
Avg Fuel/Voyage      | 10613        | 4844         | +119.1%
Avg CO2/Voyage       | 28593        | 13365        | +113.9%
Avg Efficiency       | 43.53        | 28.64        | +52.0%
------------------------------------------------------------
